In [168]:
import cv2
from facepp import API, File
import pandas as pd
import time
from pandas.io.json import json_normalize

from pandas import compat
from findtools.find_files import (find_files, Match)
import codecs
from pandas import compat
import ast, json
from pandas import DataFrame
import yaml
import numpy as np

In [85]:
#repo_path = "/Users/dev/Desktop/repos/faces/"
repo_path = "./"
#repo_path = "/home/samantha/repos/faces/"
from findtools.find_files import (find_files, Match)
import collect_facial_points
reload(collect_facial_points)
#Creating training dataset

infile_path = repo_path + 'test/'
#CFD2/images2/'
outfile_path = repo_path + 'cropped/test/'
#sh_files_pattern = Match(filetype='f', name='*N.jp*')
#sh_files_pattern1 = Match(filetype='f', name='*HO.jp*')
#found_files = list(find_files(path=infile_path, match=sh_files_pattern)) + list(find_files(path=infile_path, match=sh_files_pattern1))
sh_files_pattern = Match(filetype='f', name='*.jp*')
found_files = list(find_files(path=infile_path, match=sh_files_pattern))
data_path = repo_path + 'data/'

In [330]:
api = API('b40d3ea28aad1f2c2aa939dec674abff', '_gYS77zBAL0U60gtv_qebqwRmRWJz4c1')

#Find photos with faces and output meta photo information
def get_face_meta(found_files, infile_path, data_path, api):

    facesMeta = pd.DataFrame()
    facesMetaList = []
    print(len(found_files))
    for imgpath in found_files: 
        result = api.detection.detect(img = File(imgpath),  attribute="pose,age,gender,race")
        r_dump = json.dumps(result)
        result2=yaml.safe_load(r_dump)
        #temp = json_normalize(result['face'])
        #temp['imgpath'] = pd.concat([pd.DataFrame({'imgpath': imgpath.replace(infile_path,'')}, index = range(1))]*len(result['face']), ignore_index=True)  
        #temp['img_width'] = pd.DataFrame({'img_width': [result['img_width']]*len(result['face'])})
        #temp['img_height'] = pd.DataFrame({'img_height': [result['img_height']]*len(result['face'])}) 
        if len(result['face']) > 0:
            
                
            facesMeta = pd.concat([facesMeta, temp], axis=0, ignore_index = True)  
        
    facesMeta.to_pickle("./data/facesMeta1.pkl")
    
    return(facesMeta)

#Get facial points from each face in photos
def get_key_points(faces, api):
    allfacePts = []
    allFaces = []
    allFaces2 = pd.DataFrame()
 
    print(len([items.encode('ascii','ignore') for items in faces.ix[:,'face_id']]))
    for n,idx in enumerate([items.encode('ascii','ignore') for items in faces.ix[:,'face_id']]):
        print(idx)
        r2 = api.detection.landmark(face_id = idx, type = "83p")
        
        faces2 = json_normalize(r2["result"][0]["landmark"])
        faces2['face_id'] = idx
        #image = cv2.imread(imgpath)
        
        if allFaces2.empty == 'True':
            allFaces2 = faces2
        else:
            allFaces2 = pd.concat([allFaces2,faces2],axis=0)
        
        #faces2.merge(pd.DataFrame(face["face_id"]))
        r3 = r2["result"][0]["landmark"]
        
                
        for key,value in r3.iteritems():
            facePts = {}
            facePts["face_id"] = idx
            facePts["position"] = key.encode("ascii","ignore")
            for k,v in value.iteritems():
                if k == u'x':
                    facePts["x"] = v
                if k == u'y':
                    facePts["y"] = v
            allfacePts.append(facePts)

        finalFaces = pd.DataFrame(allfacePts)
        allFaces2.to_pickle("./data/facesPts1.pkl")
        finalFaces.to_pickle("./data/facesPtsLong1.pkl")
    return [allFaces2, finalFaces]
        
        
    
                
def mark_key_points(faces, facepts,infile_path,outfile_path):                
        #Mark key facial points on photo
    for n,idx in enumerate([items.encode('ascii','ignore') for items in faces.ix[:,'face_id']]):
        face = faces[faces['face_id']==idx]
        pts = facepts[facepts['face_id']==idx]
        
        print(infile_path + face.iloc[0]['imgpath'])
        image = cv2.imread(infile_path + face.iloc[0]['imgpath'])
        
        for row in pts.iterrows():
            cx = row[1]['x']/100*face.iloc[0]['img_width'] 
            cy = row[1]['y']/100*face.iloc[0]['img_height']
            cv2.circle(image,(int(cx),int(cy)),2,(0,0,255),1)
        
        #crop ima∫ge based on key points
        startx = max(0,((face["position.center.x"].values-0.6*face['position.width'].values)/100)*face['img_width'].values)
        starty = max((face["position.center.y"].values-1.25*face['position.height'].values)/100*face['img_height'].values,0)
        endx =((face["position.center.x"].values+0.6*face['position.width'].values)/100)*face['img_width'].values
        endy = ((face["position.center.y"].values + 0.8*face['position.height'].values)/100)*face['img_height'].values
        
        
        cropped = image[starty:endy, startx:endx]
        cv2.imshow("cropped", cropped)
        printid = outfile_path + face.iloc[0]["face_id"] +".jpeg"
        cv2.imwrite(printid, cropped)
        time.sleep(2)
        cv2.destroyAllWindows()
                
        time.sleep(1)
        
           
        #finalFaces = []    
        cv2.destroyAllWindows()


        
        

http://api.us.faceplusplus.com/


In [291]:

cv2.destroyAllWindows()
infile_path+face.iloc[0]['imgpath']

'./test/facial-expressions.jpg'

In [247]:
list(pts.iterrows())[0][1]['x']

58.444867

In [ ]:

infile_path = repo_path + 'CFD2/images2/'
#CFD2/images2/'
outfile_path = repo_path + 'cropped/CFD2/'
sh_files_pattern = Match(filetype='f', name='*N.jp*')
sh_files_pattern1 = Match(filetype='f', name='*HO.jp*')
found_files = list(find_files(path=infile_path, match=sh_files_pattern)) + list(find_files(path=infile_path, match=sh_files_pattern1))
sh_files_pattern = Match(filetype='f', name='*.jp*')
found_files = list(find_files(path=infile_path, match=sh_files_pattern))
data_path = repo_path + 'data/'

t=get_face_meta(found_files, infile_path, data_path, api)
y=get_key_points(t,api)
z=mark_key_points(t, y[1],infile_path,outfile_path)


In [326]:
x=pd.read_pickle("./data/facePtsLong.pkl")
print(x)

                               face_id                           position  \
0     d49a64d1736c9d6b9854a70534f43bbf      mouth_upper_lip_left_contour2   
1     d49a64d1736c9d6b9854a70534f43bbf                       contour_chin   
2     d49a64d1736c9d6b9854a70534f43bbf     mouth_lower_lip_right_contour3   
3     d49a64d1736c9d6b9854a70534f43bbf                     contour_right9   
4     d49a64d1736c9d6b9854a70534f43bbf      mouth_upper_lip_left_contour1   
5     d49a64d1736c9d6b9854a70534f43bbf         right_eyebrow_lower_middle   
6     d49a64d1736c9d6b9854a70534f43bbf          left_eyebrow_lower_middle   
7     d49a64d1736c9d6b9854a70534f43bbf      mouth_upper_lip_left_contour3   
8     d49a64d1736c9d6b9854a70534f43bbf    left_eyebrow_lower_left_quarter   
9     d49a64d1736c9d6b9854a70534f43bbf   right_eyebrow_lower_left_quarter   
10    d49a64d1736c9d6b9854a70534f43bbf  right_eyebrow_lower_right_quarter   
11    d49a64d1736c9d6b9854a70534f43bbf                 nose_contour_left1   

In [29]:
xx=y[1]
#print(y[0][y[0]['face_id']=='9aa60939b395a97d6df5ce731f16332d'])
t.columns

Index([             'attribute.age.range',              'attribute.age.value',
            'attribute.gender.confidence',           'attribute.gender.value',
       'attribute.pose.pitch_angle.value',  'attribute.pose.roll_angle.value',
         'attribute.pose.yaw_angle.value',        'attribute.race.confidence',
                   'attribute.race.value',                          'face_id',
                      'position.center.x',                'position.center.y',
                    'position.eye_left.x',              'position.eye_left.y',
                   'position.eye_right.x',             'position.eye_right.y',
                        'position.height',            'position.mouth_left.x',
                  'position.mouth_left.y',           'position.mouth_right.x',
                 'position.mouth_right.y',                  'position.nose.x',
                        'position.nose.y',                   'position.width',
                                    'tag',          

In [120]:
#print(t[t['face_id']=='d196d23bd058901975f412c9f028e03e'])
print(t)

    attribute.age.range  attribute.age.value  attribute.gender.confidence  \
0                     5                   20                      98.8680   
1                     5                   23                      98.1934   
2                     5                   13                      88.4721   
3                     5                   18                      80.4434   
4                     5                   19                      95.8584   
5                     5                   16                      72.0528   
6                     5                   18                      99.8605   
7                     6                   14                      93.5005   
8                     5                    8                      92.6094   
9                     5                   24                      79.5698   
10                    5                   11                      99.4605   
11                    5                   35                      90.8298   

In [21]:
file1= data_path + 'facesMeta.csv'
print(file1)
t.to_csv
#to_csv(path_or_buf='/home/samantha/repos/faces/facesMeta.csv', sep='\t', encoding="ascii")
#t.to_pickle(file1+'.pkl')
#x=pd.DataFrame.from_csv(data_path + 'facesMeta.csv',sep=',')


/home/samantha/repos/faces/data/facesMeta.csv


pandas.core.index.Index

In [31]:
y[1].columns

Index(['face_id', 'position', 'x', 'y'], dtype='object')